In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
custom_rc = {"figure.figsize": (8, 6)}
sns.set(rc=custom_rc)

In [ ]:
medical = pd.read_csv("/content/insurance.csv")
df = medical.copy()

In [ ]:
df.shape

In [ ]:
df.head(15)

In [ ]:
df.info()
df.isna().sum()

In [ ]:
df.describe().T

In [ ]:
age_category= [
    (0, 18),
    (19, 25),
    (26, 30),
    (31, 35),
    (36, 40),
    (40, 45),
    (46, 50),
    (50, 55),
    (56, 60,),
    (61, 65),
    (66, 100)
]
# Yaş gruplarına göre kategorik değişken oluşturuldu
df['age_category'] = pd.cut(df['age'], bins=[x[0] for x in age_category] + [age_category[-1][1]], labels=["0-18", "19-25", "26-30", "31-35", "36-40" , "40-45" , "46-50" , "50-55" , "56-60" , "61-65" , "66+"], right=False)

In [ ]:
bmi_category = [
    (0, 15),      #(Ciddi Aşırı Zayıf)
    (16, 18),     #(Aşırı Zayıf)
    (18.5, 24.9), # (Normal)
    (25, 29.9),   #(Kilolu)
    (30, 34.9),   #(Obez Tip 1)
    (35, 39.9),   #(Obez Tip 2)
    (40, float('inf'))  # 40+ için (Obez Tip 3)
]

# Bmi gruplarına göre kategorik değişken oluşturuldu
df['bmi_category'] = pd.cut(df['bmi'], bins=[x[0] for x in bmi_category] + [bmi_category[-1][1]], labels=["Ciddi Aşırı Zayıf" , "Aşırı Zayıf" , "Normal", "Kilolu","Obez Tip 1" , "Obez Tip 2", "Obez Tip 3"], right=False)


In [ ]:
df.head()

In [ ]:
#Veri Setinin Normal Bir Dağılım Gösterip Göstermediğini Shapiro-Wilk Testi Uygulanması
from scipy import stats
sütunlar = df[['age', 'charges', 'bmi', 'children']]
for sütun in sütunlar.columns:
    statistik, p_degeri = stats.shapiro(sütunlar[sütun])
    print(f'{sütun} - Statistik: {statistik}, p-değeri: {p_degeri}')
    alfa = 0.05
    if p_degeri > alfa:
        print(f'{sütun} için veri normal bir dağılıma sahiptir (H0 reddedilmedi)')
    else:
        print(f'{sütun} için veri normal bir dağılıma sahip değildir (H0 reddedildi)')

Shapiro Wilk testine göre veri setimizde bulunan age - charges - bmi ve children değişkenleri normal dağılım göstermiyor. Ancak ayrıca histogram ve yada Q-Q plotu ile de görselleştirerek değerlendirmek gerekiyor.

In [ ]:
import scipy.stats as stats
# Q-Q plt çizmek
stats.probplot(df.age, dist="norm", plot=plt)
plt.title("Q-Q Plot")
plt.show()


In [ ]:
stats.probplot(df.bmi, dist="norm", plot=plt)
plt.title("Q-Q Plot")
plt.show()


In [ ]:
stats.probplot(df.children, dist="norm", plot=plt)
plt.title("Q-Q Plot")
plt.show()


In [ ]:
stats.probplot(df.charges, dist="norm", plot=plt)
plt.title("Q-Q Plot")
plt.show()


Q-Q plt ile görsel olarak incelendiğinde bmi değişkeni diğerlerine oranla biraz daha iyi normal dağılım göstermektedir. Normal dağılım göstermemelerinin sebebi veri setinde aykırı değerler (outlier) olduğu için olabilir. Bunun için önce boxplot ile görselleştirme yapılarak aykırı değerlerin olup olmadığını tespit etmeliyiz.

In [ ]:
sütun_adı = 'age'
plt.figure(figsize=(8, 6))
plt.boxplot(df[sütun_adı], vert=False)
plt.title(f'{sütun_adı} Box Plot')
plt.xlabel(sütun_adı)
plt.show()

In [ ]:
sütun_adı = 'children'
plt.figure(figsize=(8, 6))
plt.boxplot(df[sütun_adı], vert=False)
plt.title(f'{sütun_adı} Box Plot')
plt.xlabel(sütun_adı)
plt.show()

In [ ]:
sütun_adı = 'bmi'
plt.figure(figsize=(8, 6))
plt.boxplot(df[sütun_adı], vert=False)
plt.title(f'{sütun_adı} Box Plot')
plt.xlabel(sütun_adı)
plt.show()

In [ ]:
sütun_adı = 'charges'
plt.figure(figsize=(8, 6))
plt.boxplot(df[sütun_adı], vert=False)
plt.title(f'{sütun_adı} Box Plot')
plt.xlabel(sütun_adı)
plt.show()

Age ve children değişeknlerinde aykırı değer görünmüyor ancak bmi ve charges değişkenlerinde aykırı değerler olduğu görülmektedir. BMİ değişkenini inceledim ve 18 altı ve 40 üstü hastaklarında olabileceği mümkün olduğu için bu değerler aykırı değer olarak ele alınmadı bu nedenle üzerlerinde herhangi bir değişiklik yapılmadı.
Charges değişkeni ise ele alınarak genel bir inceleme yapıldı.

In [ ]:
genel_ist = df.groupby('region')['charges'].describe()
genel_ist

Burada bölge bazında ödenen ücretlerin ortalama değerlerine erişebiliyoruz. Öncelikle burada dengeli bir veri olduğunu söyleyemeyiz çüngi bölge bazındaki veriler eşit değildir tabiki bu durum ortalama ve sapmayı etkileyecektir. Buraya göre youmlarsak eğer en çok southeast bölgesi ortalama diğer bölgelere göre daha çok ödeme yapmaktadır. Ancak genel olarak std değerlerini incelediğimizde bu oranlar çok yüksek olduğu görülebilmekte buda aykırı değerler içerdiğinin ipucunuda bize vermektedir.

In [ ]:
genel_ist = df.groupby('sex')['charges'].describe()
genel_ist

Erkek ve kadın sayıları aynı olmadığından hesaplamalarda değişkenlik olması muhtemeldir. Buraya göre incelediğimizde erkeklere kadınlara göre daha fazla sağlık ücreti ödemesi aldıkları görülebilmektedir.

In [ ]:
genel_ist = df.groupby('smoker')['charges'].describe()
genel_ist

Sigara içenlerin içmeyenlere göre daha fazla sağlık ücreti ödemesi yapılmıştır. Ayrıca Veri setimizdeki 1338 kişiden büyük çoğunluğu sigara içmediğide anlaşılmaktadır.

In [ ]:
genel_ist = df.groupby('children')['charges'].describe()
genel_ist

Veri setimiz dengeli olmasada(çoçuk sayısı bakımından burda) buradan anlaşıldığı üzere çoçuk sayısı arttıkça kişiye ödenen sağlık ücretinin arttığını anlayabiliriz. Çünki 1338 kişisen 18 kişi sadece 5 çoçuğa sahip olsada ortalama değeri diğerlerine göre yakın bir değer çıkmıştır.

In [ ]:
genel_ist = df.groupby('age_category')['charges'].describe()
genel_ist

Burada 1338 kişiden diğerlerine oranla en fazla 19-25 yaş aralığı kişiler vardır ve ortalama değerleri incelendiğinde Kişilerin yaş değerleri arttıkça aldıkları ssağlık ödemesi ücretlerinde artış bulunmaktadır. Ayrıca 19-25 aralığında artış diğer genç nüfüsa göre fazla olması dikkat çekicidir.

In [ ]:
genel_ist = df.groupby('bmi_category')['charges'].describe()
genel_ist

Burada bireylerin bmi değerleri arttıkça insanların aldıkları sağlık ücretlerinde artış olduğu anlaşılmaktadır.

Note: Charges değişkeninde aykırı değerler içerdiğinden yukarıda charges değişkeninne göre elde edilen bulgular çok güvenilir olmasada bizim genel bir değerlendirme yapmamızı sağlamaktadır.

In [ ]:
# Charges değişeninde bulunan aykırı değerleri tespit etme
alt_deger = np.percentile(df['charges'], 1)
ust_deger = np.percentile(df['charges'], 99)

aykiri_indeksler = (df['charges'] < alt_deger) | (df['charges'] > ust_deger)
aykiri_satirlar = df[aykiri_indeksler]

print("Aykırı Değerlerin Satır İndeksleri:")
print(aykiri_indeksler[aykiri_indeksler].index)
print("Aykırı Değerlere Sahip Satırlar:")
print(aykiri_satirlar)


In [ ]:
#Daha iyi gözlemleyebilmek için bölgeye göre bir sıralama yaptırma
aykiri_satirlar.sort_values(by=['region', 'charges'])

In [ ]:
filtre = (df['age_category'] == '50-55') & (df['region'] == 'northwest') & (df['sex'] == 'male')
sonuc_df = df[filtre]

print(sonuc_df)

Buradaki değerler incelendiğinde çok benzer olan kişilerin mesela ilk iki satırdaki bireye bakıldığında aralarındaki charges değerleri çok fazla olduğu görülmektedir. Aralarındaki tek fark sadece birinin bir çoçuğa sahip olmasıdır. Burada bu değerlerin aykırı olduğu kabul edilerek baskılama(Winsorization) yöntemi ile bölge bazında charges değerleri güncellenecektir. Bu kısım görsel işlemi tamamlandıktan sonra yapılacaktır.

Buradan sonraki kısımda görsel olarak incelenilmeye devam edilecektir. Genel olarak görsellerde ortalama charges değeri baz alınırak karşılaştırmalar yapılmıştır. Çünki bireylere ödenen sağlık ücretleri ele alınarak değişkenlerin incelenmesini daha doğru olduğu düşünülmüştür. Ayrıca charges değeri aykırı değerler içerdiği için genel olarak çıkan izlenimler doğru olmayacaktır. Bu nedenle aykırı değerler düzeltilmeden önce bir genel izlenim sahibi olmak için burada böyle yapılmıştır.

In [ ]:
mean_charges = df.groupby(['region', 'sex'])['charges'].mean().reset_index()
plt.figure(figsize=(12, 6))
sns.barplot(x='region', y='charges', hue='sex', data=mean_charges)
plt.title('Cinsiyet ve Bölgeye Göre Ortalama Charges')
plt.show()

Bölgeler bazında bakıldığında erkeklerin sayısı genel olrak kadınlardan daha fazla sadece northwest bölgesinde kadınlar bir tik fazladır. Ayrıca erkeklerin northwest dışında diğer bölgelerde daha fazla sağlık ödemesi aldıkları görülebilmektedir.

In [ ]:
mean_charges = df.groupby(['age_category', 'sex'])['charges'].mean().reset_index()
plt.figure(figsize=(12, 6))
sns.barplot(x='age_category', y='charges', hue='sex', data=mean_charges)
plt.title('Cinsiyet ve Yaş Gruplarına Göre Ortalama Charges')
plt.show()


Daha önce genel_ist kısmında 19-25 arası hastaların çok olduğunu belirtmiştik. Buradanda anlaşılacağı üzere 19-25 arası bireylerin en çok erkek hastalar oluşturmaktadır. Genel bazda yaş olarak bakıldığında erkeklerin daha fazla olduğu görülebilmektedir.

In [ ]:
mean_charges = df.groupby(['age_category', 'smoker'])['charges'].mean().reset_index()
plt.figure(figsize=(12, 6))
sns.barplot(x='age_category', y='charges', hue='smoker', data=mean_charges)
plt.title('Bölge Bazında Bireylerin Sigara Kullanım Durumlarına Göre Ortalama Charges')
plt.show()


Daha önce gelen_ist kısmında Erkeklerin kadınlara göre daha yüksek oranda sağlık ödemesi aldıkları anlaşılmıştı. Buradanda anlaşıldığı üzere erkeklerin kadınlara göre daha yüksek oranda sigara içiminden dolayı hastalandıkları görüleebilmektedir.

In [ ]:
mean_charges = df.groupby(['children', 'sex'])['charges'].mean().reset_index()
plt.figure(figsize=(12, 6))
sns.barplot(x='children', y='charges', hue='sex', data=mean_charges)
plt.title('Cinsiyet Ve Sahip Olunan Çoçuk Sayısına Göre Ortalama Charges')
plt.show()


In [ ]:
mean_charges = df.groupby(['bmi_category', 'sex'])['charges'].mean().reset_index()
plt.figure(figsize=(12, 6))
sns.barplot(x='bmi_category', y='charges', hue='sex', data=mean_charges)
plt.title('Cinsiyet Ve Bmi Kategörilerine Göre Ortalama Charges')
plt.show()


Burada bmi kategorileri değerleri incelendiğinde genel olarak erkeklerin büyük çoğunluğu Obez seviyesinde kiloludur. Kadınlar ise erkeklere göre daha az obez olsalarda aşırı zayıf olanlarıda vardır. Tabi içlerinde ciddi oranda zayıf olan kişiler sadece erkek olduğuda görülebilmektedir .

In [ ]:
mean_charges = df.groupby(['region', 'bmi_category'])['charges'].mean().reset_index()
plt.figure(figsize=(12, 6))
sns.barplot(x='region', y='charges', hue='bmi_category', data=mean_charges)
plt.title('Bölge Bazında Bmi Kategorilerine Göre Ortalama Charges')
plt.show()

Bir önceki görselde ciddi oranda aşırı zayıf olan erkeklerin sadece northeast bölgesinde yer aldığı görülebilmektedir.
Genel olarak bakıldığında en çok üst seviye obez insanların bulunduğu bölge southeast olduğu anlaşılmaktadır. Genel olarakta bireylerin yüksek oranda normalin üstünde kilolu olduklarıda görülebilmektedir.

In [ ]:
mean_charges = df.groupby(['region', 'children'])['charges'].mean().reset_index()
plt.figure(figsize=(12, 6))
sns.barplot(x='region', y='charges', hue='children', data=mean_charges)
plt.title('Bölge Bazında Sahip Olunan Çocuk Sayılarına Göre Ortalama Charges')
plt.show()

Buradaki elde edilen grafiğe göre northwest ve southeast bölgesinde en çok 2-3 çoçuk sahibi olan bireylere daha fazla sağlık ücreti ödemesi yapılırken southwest bölgesinde 2 ve 4 çoçuklu bireylere daha fazla sağlık ödemesi yapıldığı tespit edilebilmektedir. Geriye kalan northeast bölgesinde ise en çok 1 çoçuklu bireylerin daha fazla ödeme aldığı görülebilmektedir.

In [ ]:
mean_charges = df.groupby(['age_category', 'bmi_category'])['charges'].mean().reset_index()
plt.figure(figsize=(12, 6))
sns.barplot(x='age_category', y='charges', hue='bmi_category', data=mean_charges)
plt.title('Cinsiyet ve Bölgeye Göre Ortalama Charges')
plt.show()

Bu grafikte genel olarak yaş gruplarında aşırı zayıf - obez tip 3 - obez tip 2 - obez tip 1 olan kişilerin daha fazla sağlık ödemesi aldığı bilgisine ulaşılabilmektedir.

Buradan sonraki kısımda yer alan görseller projede istenilen görsel ve bilgilere yer verilmiştir.

In [ ]:
cocuk_sayisi_bolge = df.groupby('region')['children'].sum().reset_index()
print(cocuk_sayisi_bolge)

En çok çoçuk sagibi olan bölge southeast olduğu anlaşılmaktadır.

In [ ]:
#Keman Grafiği
plt.figure(figsize=(10, 6))
sns.violinplot(x='sex', y='bmi', data=df)
plt.title('BMI ve Cinsiyet İlişkisi')
plt.xlabel('Cinsiyet')
plt.ylabel('BMI')
plt.show()

Keman grafiğnde erkeklerde 50 üstü bmi değer olduğu görülebilmelte ve ayrıca her iki cinsiyette ise 20-40 bmi değere sahip bireylerden oluştuğu görülebilmektedir.

In [ ]:
#Nokta Grafiği
plt.figure(figsize=(10, 6))
sns.scatterplot(x='bmi', y='age', data=df)
plt.xlabel('BMI (Vücut Kitle İndeksi)')
plt.ylabel('Yaş')
plt.title('BMI ve Yaş Arasındaki İlişki')
plt.show()


Nokta grafiği ile bmi ve age arasındaki ilişkiyi inceleyebilmek mümkün. Burada veriler çok dağınık bir biçimde yer almaktadır. Bu nedenle kategorik bazda izlenim daha güzel ve açıklayıcı bir görsel bize sunabilir.Daha önce sütün grafiğinde inceleme yapılmıştı ama burada bu sefer age ve bmi ilişkisini kategorik bazda ısı haritası ile inceleme yapıldı.Tabi charges ele alınmıştı burada sadece age ve bmi değerleri dikkate alındı.

In [ ]:
#Isı Haritası
data_to_plot = df[['age_category', 'bmi_category']]
relationship_matrix = data_to_plot.groupby(['age_category', 'bmi_category']).size().unstack()

relationship_matrix = relationship_matrix.fillna(0)

plt.figure(figsize=(10, 8))
sns.heatmap(relationship_matrix, annot=True, fmt='d', cmap='YlGnBu')
plt.title('Yaş Kategorisi ve BMI Kategorisi İlişkisi')
plt.xlabel('BMI Kategorisi')
plt.ylabel('Yaş Kategorisi')
plt.show()


Sonuç olarak 19 ile 25 yaş aralığındaki bireylerin kilolu ve obez tip 1 olduğu görülmektedir. Aşırı zayıf kişiler 0 ile 18 yaş arasındaki bireydir. Aşırı zayıf bireyler en çok 19 - 25 yaş aralığındaki bireylerdir. Normal kilolu bireyler yine en çok 19-25 yaş aralığındaki bireylerdir. Normal üstü kilolu bireyler yine en çok 19-25 yaş aralığındaki bireylerdir. Bir sağlık sorunu olaan obez tip 1 evresindeki insanlar en çok 19-25 yaş aralığındaki bireylerdir. Obez tip 2 evresindeki bireyler ise en çok 50-55 yaş arasındaki bireylerdir. obez tip 3 evresindeki bireyler 50-55 yaş aralığındaki bireylerdir. Sonuç olarak bmi değeri çok yüksek olan hastalar en çok 50-55 yaş aralığındaki insanlarda bulunmaktadır. Ancak obez tip 1 de 19-25 arasındaki bireylerde yüksek bir değer içermektedir.

In [ ]:
# Dağılım Grafiği
plt.figure(figsize=(10, 6))
sns.scatterplot(x='bmi', y='children', data=df)
plt.title('BMI ve Çocuk Sayısı İlişkisi')
plt.xlabel('BMI')
plt.ylabel('Çocuk Sayısı')
plt.show()


Dağılım grafiği ile incelediğimizde bmi değerine göre bireylerin çoçuk sayılarını görebilmekteyiz. Bunu bmi değerini kategorik olarak incelediğimizde bize daha ayrıntılı bir detay gösterecektir.

In [ ]:
#Isı haritası
pivot_data = df.groupby(['children', 'bmi_category']).size().reset_index(name='count')
heatmap_data = pivot_data.pivot('bmi_category', 'children', 'count')
plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_data, annot=True, fmt='d', cmap='YlGnBu')
plt.title('BMI Category ve Children İlişkisi Heatmap')
plt.xlabel('Children')
plt.ylabel('BMI Category')
plt.show()

Burada bireylerin bmi değerlerine göre sahip oldukları çoçuk sayısı incelendiğinde çoçuk sayısı arttıkça özellikle obez tip 1-2-3 de birey sayısının daha az olduğunu görebilmekteyiz.

In [ ]:
#Saçılma Grafiği
plt.figure(figsize=(12, 6))
sns.scatterplot(x='bmi', y='charges', data=df)
plt.title('BMI ve Charges İlişkisi')
plt.xlabel('BMI')
plt.ylabel('Charges')
plt.show()


Saçılma grafiği ile bmi ile charges arasındaki ilişkiyi inceleyebiliriz. Aslında bu grafikte charges değişkeninin aykırı değer içerebildiği tahmin edilebilir.45-50 Bmi değere sahip bir kişinin charges ödemesi 60.000 üstünde yer almış ancak yine 45-50 arasındaki bireylerin ödemeleri daha düşük olduğu görülebilmektedir.

Daha önce Shapiro-Wilk testi ile age - children - bmi ve charges değişkenlerinin normal dağılmadığını tespit etmiştik. Ancak Q-Q PLt ile incelendiğinde sadece bmi değişkenin daha normal bir dağılım gösterdiğini tespit ettik. Biz burada Shapiro-wilk hipotez testinin daha doğru bir sonuc veridiğini kabul ettik.

Şimdi daha önce elde ettiğimiz (outlire) aykırı değerlere winsorization işlemi uygalanacaktır.

In [ ]:
#Baskılama (WİNSORİZATİON) İşlemi
df.sort_values(by='region', inplace=True)
medyan_degerler = df.groupby('region')['charges'].median()
for index, row in aykiri_satirlar.iterrows():
    region = row['region']
    yeni_deger = medyan_degerler[region]
    df.loc[index, 'charges'] = yeni_deger
print(df)


Winsorization işlemi uygulanarak bölge bazında yüksek değer içerem değerler bölgelerin medyan değeri ile güncellenmiştir.
Burada bölge bazında ayrı ayrı winsoraziton işleminin yapılmasının amacı genelde her bölge kendi kurallarına göre bünyesinde yaşayan bireylere farklı tutarlarda ödeme yapması mümkündür.

In [ ]:
# Bulduğum indeks değerlerine sahip satırları seçer
secilen_satirlar = df.loc[[22, 34, 175, 194, 281, 442, 488, 530, 543, 577, 584, 663, 681, 781, 791, 808, 819, 866, 901, 940, 1062, 1146, 1230, 1241, 1244, 1251, 1300, 1317]]

# Seçilen satırları yazdırır
print(secilen_satirlar)


Burada aykiri_satirlar içinde bulunan değerler için df içinde bölge bazında bir sıralama yapılıp daha sonra medyan değerleri belirlenip tekrardan aykiri satırlar içinde yer alan bölgelere göre her bölgenin kendi medyan değeri bu değerler ile güncellendi. Aslında bu şekilde boxplot çiziminde bize aykırı değer olarak göstersede biz her bölgenin kendi ödeme tutarı belirlediğinden sadece bölge bazında güncelleme yaptıık.

In [ ]:
genel_ist = df.groupby('region')['charges'].describe()
genel_ist


          	count        mean

northeast	324.0	13406.384516      
northwest	325.0	12417.575374	    
southeast	364.0	14735.411438	  
southwest	325.0	12346.937377

daha önceki bulduğumuz değerler ile yeni değerleri karşılaştırabilirsiniz. Artık aykırı değerler olmadığı için(bölge bazında) yorum yapmak daha kolaydır.

MAKİNE ÖĞRENMESİ İLE BİREYLERİN CHARGES DEĞERİNİ TAHMİN ETME


In [ ]:
df['sex'] = df['sex'].apply(lambda x: 1 if x == 'female' else 0)

In [ ]:
df['smoker'] = df['smoker'].apply(lambda x: 1 if x == 'yes' else 0)

In [ ]:
df['region'] = df['region'].map({'southeast': 1, 'southwest': 2, 'northeast': 3, 'northwest': 4})

In [ ]:
df['bmi_category'] = df['bmi_category'].map({'Obez Tip 1': 1, 'Obez Tip 2': 2, 'Obez Tip 3': 3, 'Kilolu': 4 , 'Normal':5 , 'Aşırı Zayıf': 6 , 'Ciddi Aşırı Zayıf' : 7})

In [ ]:
df['age_category'] = df['age_category'].map({'19-25': 1, '50-55': 2, '40-45': 3, '26-30': 4 , '31-35':5 , '56-60': 6 , '46-50' : 7 , '36-40':8 , '61-65': 9 , '0-18' : 10 , '66+': 11})

In [ ]:
# Veri setini yedek alma ve bağımlı ve bağımsız değişkenler olarak ayırma yapma
orjn = df.loc[:, ["sex", "children", "smoker", "region", "charges", "age_category", "bmi_category"]]
x = orjn.drop(columns=['charges'])
y = orjn.loc[: , ["charges"]]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso,
    ElasticNet,
    PoissonRegressor,
    BayesianRidge,
    LassoLars,
    OrthogonalMatchingPursuit,
    RANSACRegressor,
    HuberRegressor,
)
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    AdaBoostRegressor,
    BaggingRegressor,
)
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.svm import SVR, NuSVR, LinearSVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Modelleri ve isimlerini bir liste içinde tutuldu
models = [
    ("Linear Regression", LinearRegression()),
    ("Ridge", Ridge()),
    ("Lasso", Lasso()),
    ("ElasticNet", ElasticNet()),
    ("Poisson Regressor", PoissonRegressor()),
    ("Bayesian Ridge", BayesianRidge()),
    ("LassoLars", LassoLars()),
    ("OMP", OrthogonalMatchingPursuit()),
    ("RANSAC", RANSACRegressor()),
    ("Huber", HuberRegressor()),
    ("Decision Tree", DecisionTreeRegressor()),
    ("Random Forest", RandomForestRegressor()),
    ("Gradient Boosting", GradientBoostingRegressor()),
    ("AdaBoost", AdaBoostRegressor()),
    ("Bagging", BaggingRegressor()),
    ("KNN", KNeighborsRegressor()),
    ("NuSVR", NuSVR()),
    ("Linear SVR", LinearSVR()),
    ("LightGBM", LGBMRegressor()),
]


In [ ]:
results_df = pd.DataFrame(columns=["Model", "MAE", "MSE", "MAPE", "RMSE", "R-squared"])

for name, model in models:
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r_squared = r2_score(y_test, y_pred)

    results_df = results_df.append({
        "Model": name,
        "MAE": mae,
        "MSE": mse,
        "MAPE": mape,
        "RMSE": rmse,
        "R-squared": r_squared
    }, ignore_index=True)

print(results_df)

<ipython-input-80-9f877f1cd55d>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-80-9f877f1cd55d>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-80-9f877f1cd55d>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-80-9f877f1cd55d>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000393 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 32
[LightGBM] [Info] Number of data points in the train set: 936, number of used features: 6
[LightGBM] [Info] Start training from score 13009.542463
                Model          MAE           MSE      MAPE          RMSE  \
0   Linear Regression  5289.246975  4.624223e+07  0.789134   6800.164065   
1               Ridge  5293.989513  4.627603e+07  0.791561   6802.648461   
2               Lasso  5289.459046  4.624318e+07  0.789286   6800.234156   
3          ElasticNet  7175.360300  8.812705e+07  1.177509   9387.600639   
4   Poisson Regressor  4877.672711  4.118195e+07  0.777160   6417.316143   
5      Bayesian Ridge  5292.463886  4.626509e+07  0.790815   6801.844959   
6           LassoLars  5289.459036  4.624318e+07  0.789286  

<ipython-input-80-9f877f1cd55d>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-80-9f877f1cd55d>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-80-9f877f1cd55d>:13: FutureWarning: The frame.appen

Burada en iyi algoritma 12 numara olan Gradient BOOsting Algoritmasının  kullanılması şuan için uyguun görülmektedir. Hem R-squared değeri(y nin varyasyonunu en iyi açıklaması) hemde mae değeri en düşük olması hemde rmse değeri düşük olması nedeniye bu algoritma seçimi doğru olacaktır.

Şimdi Seçilen Gradient Boosting Algoritması için en iyi cross validation değerini belirleyerek modelimizin doğru tahmin gücünü arttırmaya çalışalım.

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
#En iyi cv değerini bulmak
veri = pd.DataFrame(orjn)
hedef = veri['charges']

model = GradientBoostingRegressor()

cross_validation_numbers = [3, 5, 10, 15]
results = []

for cv_num in cross_validation_numbers:
    scores_mae = -cross_val_score(model, veri, hedef, cv=cv_num, scoring='neg_mean_absolute_error')
    scores_mse = -cross_val_score(model, veri, hedef, cv=cv_num, scoring='neg_mean_squared_error')

    mape_scores = []
    r_squared_scores = []
    for train_idx, test_idx in KFold(n_splits=cv_num).split(veri):
        veri_train, veri_test = veri.iloc[train_idx], veri.iloc[test_idx]
        hedef_train, hedef_test = hedef.iloc[train_idx], hedef.iloc[test_idx]
        model.fit(veri_train, hedef_train)
        hedef_pred = model.predict(veri_test)
        mape = np.mean(np.abs((hedef_test - hedef_pred) / hedef_test)) * 100
        mape_scores.append(mape)
        r_squared = model.score(veri_test, hedef_test)
        r_squared_scores.append(r_squared)

    scores_rmse = np.sqrt(scores_mse)


    result = {
        "Çapraz Doğrulama Sayısı": cv_num,
        "MAE": scores_mae,
        "MSE": scores_mse,
        "RMSE": scores_rmse,
        "MAPE": mape_scores,
        "R-squared": r_squared_scores
    }

    results.append(result)

for result in results:
    print(f"Çapraz Doğrulama Sayısı: {result['Çapraz Doğrulama Sayısı']}")
    print(f"MAE: {np.mean(result['MAE'])}")
    print(f"MSE: {np.mean(result['MSE'])}")
    print(f"RMSE: {np.mean(result['RMSE'])}")
    print(f"MAPE: {np.mean(result['MAPE'])}")
    print(f"R-squared: {np.mean(result['R-squared'])}")

Burada en iyi cv değeri 15 olarak görülmekte ancak modelimizi overfitting yaptığı için bu sefer üzerinde bir L1 veya L2 ile regularization işlemi yapılması uygun olacaktır.

In [ ]:
#L1 Regresyon
veri = pd.DataFrame(orjn)
hedef = veri['charges']
x = veri.drop(columns=['charges'])

x_train, x_test, y_train, y_test = train_test_split(x, hedef, test_size=0.3, random_state=42)

model = GradientBoostingRegressor()
lasso = Lasso(alpha=0.01)  # alpha değerini değğiştirebiliriz

lasso.fit(x_train, y_train)

selected_features = x_train.columns[lasso.coef_ != 0]
model.fit(x_train[selected_features], y_train)


y_pred = model.predict(x_test[selected_features])


mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
r_squared = model.score(x_test[selected_features], y_test)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAPE: {mape}")
print(f"R-squared: {r_squared}")


In [ ]:
#L2 Regresyon
hedef = veri['charges']
x = veri.drop(columns=['charges'])


x_train, x_test, y_train, y_test = train_test_split(x, hedef, test_size=0.3, random_state=42)

#L2 modeli kuruldu
ridge = Ridge(alpha=0.01)
ridge.fit(x_train, y_train)
selected_features = x_train.columns

model = GradientBoostingRegressor()
model.fit(x_train[selected_features], y_train)

y_pred = model.predict(x_test[selected_features])
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
r_squared = model.score(x_test[selected_features], y_test)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAPE: {mape}")
print(f"R-squared: {r_squared}")


In [ ]:
# en iyi hiperparametre değerlerini belirlemek
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [25,50, 75,100, 200],
    'learning_rate': [0.01, 0.1, 0.2,0.3],
    'max_depth': [2,3, 4, 5],

}

grid_search = GridSearchCV(GradientBoostingRegressor(), param_grid, cv=15, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1) #cv 15 olarak belirlenmişti
grid_search.fit(x_train, y_train)
print("En iyi hiperparametreler:", grid_search.best_params_)
print("En iyi sonuç (RMSE):", -grid_search.best_score_)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(x_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r_squared = r2_score(y_test, y_pred)

print("MAE:", mae)
print("R-squared:", r_squared)
print("MSE:", mse)
print("MAPE:", mape)


Sonuç olarak L1 veya L2 ile çalışşsak dahi modelimizi ilk kurup çalıştırdığımızda elde ettiğimiz model başarı metrik değerlerinden bir farkı olmadı. Daha sonra modelin cv değeri 15 olarak bu sefer modelin hiper parametre değerleri(sadece n_estimators - learning rate ve max_dept parametreleri alındı) ele alınarak en iyi hiper parametre değerleri belirlenerek en iyi başarı metrik değerleri tespit edinilmeye çalışıldı.
Sonucunda modelin en iyi hiper parametreleri :
n_estimators  : 100
learning rate : 0.2
max_dept      : 2 olarak bulundu.

Bu model ele alınırsa modelimiz %76 oranında doğru tahmin yapabilir diyebiliriz.


Bu kısımda Gradient Boosting Algoritması dışında diğer en iyi sonucu veren modeller ile de çalışarak Gradient Boosting Algoritmasından daha iyi bir sonuç elde edilenebilir olması mümkün olduğundan işlemler tekrarlandı ve (LightGBM , Random Forest - Bagging - Adabost - Poisson Regressor Modelleri) sonucunda Gradient Boosting algoritmasından daha iyi çalışan model LightGBM oldu.

LightGBM  modeli default olarak çalıştığında model başarısı %75 den %77 oranında artış yapılarak Gradient Boosting Modeline göre daha doğru tahmin yapabilen bir model oldu.